In [ ]:
import polars as pl
import numpy as np
import holoviews as hv
from time import time

from cow_amm_trades import compute_cow_amm_trades
from balancer_pool_states import compute_balancer_pool_states
from coingecko import get_historical_token_price
from dataframes import (
    states_to_df,
    prices_to_df,
    combine_with_prices,
    plot_profit_vs_holding,
    compute_profit_vs_holding_apy,
)
from constants import (
    TOKEN0,
    TOKEN1,
    TOKEN0_COINGECKO,
    TOKEN1_COINGECKO,
)

In [ ]:
cow_amm_states = compute_cow_amm_trades()
balancer_pool_states = compute_balancer_pool_states()

In [ ]:
df_cow = states_to_df(cow_amm_states)
df_balancer = states_to_df(balancer_pool_states)

In [ ]:
token0_prices = get_historical_token_price(TOKEN0_COINGECKO)
token1_prices = get_historical_token_price(TOKEN1_COINGECKO)
token0_price_in_eth = get_historical_token_price(TOKEN0_COINGECKO, currency="eth")
token1_price_in_eth = get_historical_token_price(TOKEN1_COINGECKO, currency="eth")

df_token0_prices = prices_to_df(token0_prices)
df_token1_prices = prices_to_df(token1_prices)
df_token0_prices_in_eth = prices_to_df(token0_price_in_eth)
df_token1_prices_in_eth = prices_to_df(token1_price_in_eth)

In [ ]:
from constants import START_TIME
from constants import START_TIME, PROTOCOL_FEE_WEI

time_now = time()
df_cow_filtered = df_cow.filter(pl.col("time") >= START_TIME)
df_balancer_filtered = df_balancer.filter(pl.col("time") >= START_TIME)

In [ ]:
df_cow_aggregate = combine_with_prices(df_cow_filtered, df_token0_prices, df_token1_prices)
df_balancer_aggregate = combine_with_prices(df_balancer_filtered, df_token0_prices, df_token1_prices)

In [ ]:
profit_vs_holding_cow = compute_profit_vs_holding_apy(df_cow_aggregate)
print(f"APY of CoW AMM: {profit_vs_holding_cow}")

protocol_fee_correction = (
    1
    + PROTOCOL_FEE_WEI
    / 10**18
    * df_cow_aggregate["price_"+TOKEN0][-1]
    / df_cow_aggregate["total_value"][-1]
)
profit_vs_holding_cow = compute_profit_vs_holding_apy(
    df_cow_aggregate, protocol_fee_correction
)
print(f"APY of CoW AMM (without paying protocol fees): {profit_vs_holding_cow}")

profit_vs_holding_balancer = compute_profit_vs_holding_apy(df_balancer_aggregate)
print(f"APY of Balancer AMM: {profit_vs_holding_balancer}")

In [ ]:
trade_fraction = len(df_cow_filtered) / len(df_balancer_filtered)
volume_fraction = (
    df_cow_aggregate.with_columns(
        pl.col("value_"+TOKEN0).diff().abs().sum().alias("volume")
    )["volume"][0]
    / df_balancer_aggregate.with_columns(
        pl.col("value_"+TOKEN0).diff().abs().sum().alias("volume")
    )["volume"][0]
)
tvl_fraction = (
    df_cow_aggregate["total_value"][-1] / df_balancer_aggregate["total_value"][-1]
)
print(f"number of trades of CoW AMM vs balancer {trade_fraction * 100:2.1f}%")
print(f"volume of CoW AMM vs balancer {volume_fraction * 100:2.1f}%")
print(f"TVL of CoW AMM vs balancer {tvl_fraction * 100:2.1f}%")

In [ ]:
plt1 = plot_profit_vs_holding(df_cow_aggregate).relabel("cow amm")
plt2 = plot_profit_vs_holding(df_balancer_aggregate).relabel("balancer")
(plt1 * plt2).opts(legend_position="top_left", title="profit vs holding comparison")

In [ ]:
plt1 = (
    df_token1_prices_in_eth.filter(pl.col("time") >= START_TIME)
    .with_columns(pl.from_epoch("time", time_unit="s"))
    .sort("time")
    .plot(x="time", y="price", label="price of "+TOKEN1+" in ETH")
)
plt2 = (
    df_token1_prices_in_eth.filter(pl.col("time") >= START_TIME)
    .with_columns(pl.from_epoch("time", time_unit="s"))
    .sort("time")
    .with_columns(
        pl.col("price")
        .rolling_var(by="time", window_size="12h")
        .alias("price_variance")
    )
    .plot(x="time", y="price_variance", label="price variance of "+TOKEN1+" in ETH")
)
plt1 + plt2